# 1. Imports

In [1]:
from sklearn.decomposition import PCA
import matplotlib as mpl 
import numpy as np
import os
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import f1_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler


from utils import stratified_train_test_group_kfold
from utils import model_test_classification

from classification_codes import knc_gridsearch



# 2. Data

In [2]:
with open("Data/MoS2_Analysis_Data_trained2", "rb") as fp:   # Unpickling
    MoS2_Proj1_Class_Data = pickle.load(fp)

#features = MoS2_ImageNet_100_data[0]
df = pd.DataFrame(MoS2_Proj1_Class_Data)
print(df.keys())
T_dict= {900:0, 950:1, 1000:2}
T_target = np.array([T_dict[item] for item in df['T']])

features = np.array(list(df['ImageNet']))
sampleId = np.array(list(df['sampleId']))

X = features
Y = np.array(T_target)
groups = np.array(sampleId)


train_val_groups, train_val_X, train_val_Y, test_X, test_Y = stratified_train_test_group_kfold(X, Y, groups, n_splits=10, test_fold=0)

train_val_Y = train_val_Y.flatten()
test_Y = test_Y.flatten()

print(train_val_X.shape)
print(train_val_Y.shape)
print(test_X.shape)
print(test_Y.shape)


oversample = RandomOverSampler(sampling_strategy='not majority')

scaler = StandardScaler().fit(train_val_X)
train_val_X = scaler.transform(train_val_X)
test_X = scaler.transform(test_X)

Index(['sampleId', 'sampleLabel', 'image', 'T', 'ImageNet', 'MicroNet'], dtype='object')


# 3. Runing Cross-Validation

In [5]:
N_neighbors = [i for i in range(3, 15)]
Weights = ['uniform', 'distance']
Algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
P = [i for i in range(1, 6)]


def cross_10_folds_knn(train_val_X, train_val_Y):
    
    best_train = []
    best_val = []
    best_variables = []
    for fold in range(10):
        
        model_path = f'classification/ImageNet/aug4/KNC_{fold}_model.sav'
        group, train_X, train_Y, val_X, val_Y = stratified_train_test_group_kfold(train_val_X, train_val_Y, train_val_groups, n_splits=10, test_fold=fold)
        
        #train_X, train_Y = oversample.fit_resample(train_X, train_Y)
        variables_best, performance_best = knc_gridsearch(train_X, train_Y, val_X, val_Y, N_neighbors, Weights, Algorithm, P, fold, model_path)
        best_train.append(performance_best['accuracy_train'])
        best_val.append(performance_best['accuracy_val'])
        best_variables.append(variables_best)
        print(f'fold: {fold} done!')
    return best_train, best_val, best_variables


best_train, best_val, best_variables = cross_10_folds_knn(train_val_X, train_val_Y)
print(best_train)
print(best_val)
print(f'train_acc_mean: {np.mean(best_train)}, std: {np.std(best_train)}, val_acc_mean: {np.mean(best_val)}, std: {np.std(best_val)}')

<class 'generator'>
best_train_acc: 0.801, best_val_acc: 0.625
best_train_acc: 0.810, best_val_acc: 0.708
best_train_acc: 1.000, best_val_acc: 0.750
best_train_acc: 1.000, best_val_acc: 0.792
fold: 0 done!
<class 'generator'>
best_train_acc: 0.782, best_val_acc: 0.667
best_train_acc: 0.791, best_val_acc: 0.708
best_train_acc: 0.806, best_val_acc: 0.750
best_train_acc: 0.749, best_val_acc: 0.792
fold: 1 done!
<class 'generator'>
best_train_acc: 0.796, best_val_acc: 0.708
best_train_acc: 0.796, best_val_acc: 0.750
best_train_acc: 1.000, best_val_acc: 0.792
fold: 2 done!
<class 'generator'>
best_train_acc: 0.791, best_val_acc: 0.667
best_train_acc: 0.825, best_val_acc: 0.750
best_train_acc: 0.749, best_val_acc: 0.792
best_train_acc: 0.744, best_val_acc: 0.833
fold: 3 done!
<class 'generator'>
best_train_acc: 0.801, best_val_acc: 0.750
best_train_acc: 1.000, best_val_acc: 0.792
best_train_acc: 1.000, best_val_acc: 0.833
fold: 4 done!
<class 'generator'>
best_train_acc: 0.830, best_val_acc:

In [9]:
import numpy as np
from sklearn.metrics import f1_score

X, Y = train_val_X, train_val_Y
groups = train_val_groups
def f1score_fn(X, Y):

    F1_score_train = []
    F1_score_val = []
    for fold in range(10):
        
        group, train_X, train_Y, val_X, val_Y = stratified_train_test_group_kfold(X, Y, groups, n_splits=10, test_fold=fold)
        #train_X, train_Y = oversample.fit_resample(train_X, train_Y)
        #model_path = f'classification/ImageNet/aug4/KNN_{fold}_model.sav'
        model_path =  f'classification/ImageNet/aug4/KNC_{fold}_model.sav'
        
        PATH = os.path.join('Models', model_path)
        
        loaded_model = pickle.load(open(PATH, 'rb'))

        pred_val_Y = loaded_model.predict(val_X)
        pred_train_Y = loaded_model.predict(train_X)

        f_score_train = f1_score(train_Y, pred_train_Y, average='macro')#'weighted')
        f_score_val = f1_score(val_Y, pred_val_Y, average='macro')#'weighted')

        F1_score_train.append(f_score_train)
        F1_score_val.append(f_score_val)
    
    return F1_score_train, F1_score_val



F1_score_train, F1_score_val = f1score_fn(X, Y)


print(f'f1_train: {np.mean(F1_score_train)}, std: {np.std(F1_score_train)}, f1_val: {np.mean(F1_score_val)}, std: {np.std(F1_score_val)}')


<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
f1_train: 0.772487404503787, std: 0.15218411113334218, f1_val: 0.6379573367808662, std: 0.10329961366609576
